In [1]:
# hey guys good morning
# today I'm going to iterate the script that reads all the informations related to a match
# for all the matches for all the major series for all the season
# let's start
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup


In [2]:
def clean_json_data(json_string_with_data):
    data = []
    for message in json_string_with_data:
        ind_start = message.index("('")+2
        ind_end = message.index("')")
        json_data = message[ind_start:ind_end]
        json_data = json_data.encode('utf8').decode('unicode_escape')
        data.append(json.loads(json_data))
    return data

In [4]:
# let's see for a given series for a given year where are stored the id for each match in that season
url = 'https://understat.com/league/Serie_A/2018'
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")
# ok just in this link there are tons of data, cool!
# the available json data are stored in the tag script named datesData, teamsData and playersData
# let's find and clean all of this data
scripts = soup.find_all('script')
json_string_with_data = []
# got it, now the json data are located only under the variable name shotsData and under rostersData
# add those strings to a list of strings
for el in scripts:
    if 'datesData' in el.text:
        json_string_with_data.append(el.text.strip())
    if 'teamsData' in el.text:
        json_string_with_data.append(el.text.strip())
    if 'playersData' in el.text:
        json_string_with_data.append(el.text.strip())
# finally let's do some magic in order to get a clean json file with the data, so that we can 
# easily understand where the interesting data are
# let's do that in a function, beause it is something that we always do
data = clean_json_data(json_string_with_data)
print(data[0]) # it is huge
# ok found!
# the list of all the match ids is in the data[0] so it means in the script file called datesData
# now the only part that is missing is to extract all the match ids in a list


IndexError: list index out of range

In [9]:
# ok so data is an array with only an element, extract that one and then, foreach entry in the 
# data[0] extract the columns and the values and store those in a data set
for i, item in enumerate(data[0]):
    print(data[0][i])
    break
# this works, now let's iterate for all the items
columns = list(data[0][0].keys())
values = []
for i, item in enumerate(data[0]):
    values.append(list(data[0][i].values()))
# ok ready to dreate the dataset with pandas
df = pd.DataFrame(values, columns=columns)
print(df.head())
# works perfectly, let's now just extract the first column called id, that is the list of all the matches
# in a season in which itherate for the http requests
df = df[df.isResult == False]
print(df.head())
matches = df['id']
print(matches)

{'id': '13089', 'isResult': True, 'h': {'id': '230', 'title': 'Parma Calcio 1913', 'short_title': 'PAR'}, 'a': {'id': '98', 'title': 'Juventus', 'short_title': 'JUV'}, 'goals': {'h': '0', 'a': '1'}, 'xG': {'h': '0.264026', 'a': '1.88829'}, 'datetime': '2019-08-24 17:00:00', 'forecast': {'w': '0.0307', 'd': '0.1393', 'l': '0.83'}}
      id  isResult                                                  h  \
0  13089      True  {'id': '230', 'title': 'Parma Calcio 1913', 's...   
1  13090      True  {'id': '110', 'title': 'Fiorentina', 'short_ti...   
2  13091      True  {'id': '99', 'title': 'Udinese', 'short_title'...   
3  13092      True  {'id': '94', 'title': 'Verona', 'short_title':...   
4  13093      True  {'id': '95', 'title': 'Roma', 'short_title': '...   

                                                   a                 goals  \
0  {'id': '98', 'title': 'Juventus', 'short_title...  {'h': '0', 'a': '1'}   
1  {'id': '105', 'title': 'Napoli', 'short_title'...  {'h': '3', 'a': '4'